## Visualizations of User Data
This notebook contains the code for the visualization featured in the presentation.

In [ ]:
# Importing and Cleaning Data
import pandas as pd
import numpy as np
import os
from random import randint
from modules.lavenshtein import lavenshtein
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_theme(style="whitegrid")

# Cleaning User Data
users_data = pd.read_csv("data/original/Users.csv")

# Dates
date_series = []
temp_date_df = users_data["date"].copy()
temp_date_df = temp_date_df.str.split("-")
for date in temp_date_df:
    length_lst = np.array(list(map(lambda x: len(x), date))) # Get str lengths
    year = date[np.argmax(length_lst)]
    # Construct new year
    if int(year[-1]) in [6, 7, 8]:
        year = "201" + str(year[-1])
    else:
        year = "201" + str(randint(6,8))
    if int(date[1]) > 12: # Check where day and month are
        if int(date[2]) > 12:
            day = date[1]
            month = str(randint(1,12)) # There's few enough cases that this should be alright
        else:
            day = date[1]
            month = date[2]
    else:
        day = date[2]
        month = date[1]
    # Construct new day
    if int(day) > 31:
        day = str(randint(1,2)) + str(day[-1])
    # Construct new date and add to users_data
    new_date = year + "-" + month + "-" + day
    date_series.append(new_date)
users_data["date"] = pd.to_datetime(date_series)

def laven_calc(cand_name, cand_lst, p_insert=1, p_delete=1, p_edit=1):
    laven_score = lavenshtein(cand_lst.iloc[0], cand_name, p_insert=p_insert, p_delete=p_delete, p_edit=p_edit)
    laven_score /= len(cand_name) # Normalize score based on string length
    return cand_lst[cand_lst == cand_name].index[0], laven_score

temp_vname_df = users_data["vendor_name"].copy() # Return List
candidate_list = users_data["vendor_name"].copy() # Iteration List
while len(candidate_list) > 0:
    temp_cand_array = candidate_list.copy()
    lavens = temp_cand_array.apply(laven_calc, cand_lst=temp_cand_array, p_insert=0.5, p_delete=0.5, p_edit=1)
    matches = list(filter(lambda x: x[1] < 0.4, lavens)) # Consider scores where less than 40% of word is edited
    matches = list(dict.fromkeys(matches)) # Remove weird duplicates
    replace_name = candidate_list.iloc[0] # Get name to replace matches with
    remove_idx = list(map(lambda x: x[0], matches))
    temp_vname_df.iloc[remove_idx,] = replace_name # Replace entries
    candidate_list = candidate_list.drop(remove_idx) # Remove matches from candidates
    candidate_list = candidate_list[~candidate_list.isin([replace_name])] # Remove matches from candidates
    print(len(candidate_list))
users_data["vendor_name"] = temp_vname_df

# Cleaning Vendor Addresses
temp_vadd_df = users_data["vendor_address"].copy() # Return List
candidate_list = users_data["vendor_address"].copy() # Iteration List
while len(candidate_list) > 0:
    temp_cand_array = candidate_list.copy()
    lavens = temp_cand_array.apply(laven_calc, cand_lst=temp_cand_array, p_insert=0.5, p_delete=0.5, p_edit=1)
    matches = list(filter(lambda x: x[1] < 0.1, lavens)) # Consider scores where less than 10% of str is different
    matches = list(dict.fromkeys(matches)) # Remove weird duplicates
    replace_name = candidate_list.iloc[0] # Get name to replace matches with
    remove_idx = list(map(lambda x: x[0], matches))
    temp_vname_df.iloc[remove_idx,] = replace_name # Replace entries
    candidate_list = candidate_list.drop(remove_idx) # Remove matches from candidates
    candidate_list = candidate_list[~candidate_list.isin([replace_name])] # Remove matches from candidates
    print(len(candidate_list))
users_data["vendor_address"] = temp_vadd_df

# Cleaning Amounts
temp_amount_df = users_data["amount"].copy() # Return List
# Impute mean of company's amounts into missing value
temp_amount_df.loc[temp_amount_df.isna()] = temp_amount_df.loc[users_data["vendor_name"] == users_data.loc[419,"vendor_name"]].mean()
users_data["amount"] = temp_amount_df

In [ ]:
temp_vname_df = users_data["vendor_name"].copy() # Return List
candidate_list = users_data["vendor_name"].copy() # Iteration List
while len(candidate_list) > 0:
    temp_cand_array = candidate_list.copy()
    lavens = temp_cand_array.apply(laven_calc, cand_lst=temp_cand_array, p_insert=0.5, p_delete=0.5, p_edit=1)
    matches = list(filter(lambda x: x[1] < 0.4, lavens)) # Consider scores where less than 40% of word is edited
    matches = list(dict.fromkeys(matches)) # Remove weird duplicates
    replace_name = candidate_list.iloc[0] # Get name to replace matches with
    remove_idx = list(map(lambda x: x[0], matches))
    temp_vname_df.iloc[remove_idx,] = replace_name # Replace entries
    candidate_list = candidate_list.drop(remove_idx) # Remove matches from candidates
    candidate_list = candidate_list[~candidate_list.isin([replace_name])] # Remove matches from candidates
    print(len(candidate_list))
users_data["vendor_name"] = temp_vname_df

# Cleaning Vendor Addresses
temp_vadd_df = users_data["vendor_address"].copy() # Return List
candidate_list = users_data["vendor_address"].copy() # Iteration List
while len(candidate_list) > 0:
    temp_cand_array = candidate_list.copy()
    lavens = temp_cand_array.apply(laven_calc, cand_lst=temp_cand_array, p_insert=0.5, p_delete=0.5, p_edit=1)
    matches = list(filter(lambda x: x[1] < 0.1, lavens)) # Consider scores where less than 10% of str is different
    matches = list(dict.fromkeys(matches)) # Remove weird duplicates
    replace_name = candidate_list.iloc[0] # Get name to replace matches with
    remove_idx = list(map(lambda x: x[0], matches))
    temp_vname_df.iloc[remove_idx,] = replace_name # Replace entries
    candidate_list = candidate_list.drop(remove_idx) # Remove matches from candidates
    candidate_list = candidate_list[~candidate_list.isin([replace_name])] # Remove matches from candidates
    print(len(candidate_list))
users_data["vendor_address"] = temp_vadd_df

In [ ]:
# Visualizing Data
plt.figure(figsize=(10, 5))
hist_chart = sns.histplot(users_data["amount"].loc[users_data["amount"]<200], kde=True, line_kws={"lw":5}, color="#017180")
plt.xlabel("Transaction Amount ($)")
plt.ylabel("# of Transactions")
plt.title("Distribution of Transaction Amounts")
plt.savefig('amount_histogram.png', bbox_inches='tight', dpi=300)
plt.show(hist_chart)

In [ ]:
# Get figure values for animated graph
hist_height = []
hist_x = []
for p in hist_chart.patches:
    hist_height.append(p.get_height())
    hist_x.append(p.get_x())
print(hist_height, hist_x)
hist_height = pd.Series(hist_height)
hist_height.to_csv("hist_height.csv")

In [ ]:
sns.violinplot(data=users_data.loc[users_data["vendor_name"] == "FOUR QUARTERS SDN BHD"], x="vendor_name", y="amount")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
unique_count = users_data["vendor_name"].value_counts().reset_index()
per_chart = sns.barplot(data=unique_count.loc[unique_count["vendor_name"] > 1], x="index", y="vendor_name", palette="Spectral")
plt.xticks([])
plt.title("Number of Transaction per Vendor")
plt.xlabel("Vendor")
plt.ylabel("# of Transactions")
#plt.savefig('amount_by_vendor.png', bbox_inches='tight', dpi=300)
plt.show(per_chart)

In [ ]:
# Get figure values for animated graph
per_height = []
per_x = []
for p in per_chart.patches:
    per_height.append(p.get_height())
    per_x.append(p.get_x())
print(per_height, per_x)
per_height = pd.Series(per_height)
per_height.to_csv("per_height.csv")

In [ ]:
unique_count = users_data["vendor_name"].value_counts().reset_index()
unique_count.loc[unique_count["vendor_name"] > 1]

In [ ]:
# Checks number of locations per vendor
leng_lst = []
for vendor in users_data["vendor_name"]:
    print(vendor)
    new_leng = len(users_data["vendor_address"].loc[users_data["vendor_name"] == vendor].value_counts())
    print(new_leng)

In [ ]:
# Checks number of vendors per location
leng_lst = []
for address in users_data["vendor_address"]:
    print(address)
    new_leng = users_data["vendor_name"].loc[users_data["vendor_address"] == address].value_counts()
    print(new_leng)